## 만개의 레시피 https://www.10000recipe.com/recipe/list.html
## 데이터 수집 코드

### url은 특정 카테고리를 선택하지 않은 "전체" 데이터 기준

In [1]:
import urllib3
urllib3.disable_warnings()

In [2]:
import pandas as pd
import requests
import time
from tqdm import trange
from bs4 import BeautifulSoup as bs

In [3]:
def get_url(page_no=1):
    url = f'https://www.10000recipe.com/recipe/list.html?order=reco&page={page_no}'
    return url

In [4]:
def get_last_page_no():
    url = get_url()
    response = requests.get(url, verify=False)
    soup = bs(response.text)
    tmp = soup.select('#contents_area_full > ul > div > b')[0].text
    last_page = int(tmp.replace(',', '')) // 40 + 1
    return last_page

In [5]:
def get_one_page(page_no):
    "한 페이지 수집"
    url = get_url(page_no)
    response = requests.get(url, verify=False)
    soup = bs(response.text)
    a_list = soup.select('#contents_area_full > ul > ul > li > div.common_sp_thumb > a')
    link_list = ['http://10000recipe.com' + a['href'] for a in a_list]
    titles = soup.select('#contents_area_full > ul > ul > li > div.common_sp_caption > div.common_sp_caption_tit.line2')
    title_list = [title.text for title in titles]
    tmp = pd.DataFrame({'요리':title_list, '링크':link_list})

    return tmp

In [6]:
def get_all_pages():
    "모든 페이지 수집"
    # last_page = get_last_page_no()
    last_page = 10  # 테스트용 10페이지 지정, 마지막페이지까지 수집하려면 윗줄 실행
    page_list = []
    # 1페이지부터 last까지
    for page in trange(1, last_page+1):
        df_one_page = get_one_page(page)
        page_list.append(df_one_page)
        time.sleep(0.01)
    
    df_all_page = pd.concat(page_list, ignore_index=True)
    return df_all_page

In [7]:
# 테스트 : 1~10페이지까지 총 400개 요리 수집
# 20230427 기준 요리 198,261개가 존재하므로 소수로 테스트
all_page = get_all_pages()
all_page

100%|██████████| 10/10 [00:15<00:00,  1.59s/it]


,요리,링크
0,오이 좋아한다면 통오이김밥,http://10000recipe.com/recipe/7001234
1,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,http://10000recipe.com/recipe/6912734
2,아삭아삭한 고추된장박이,http://10000recipe.com/recipe/4030952
3,달콤 짭조름♡ 떡 베이컨 간장조림♡,http://10000recipe.com/recipe/6843136
4,"술술 들어가는 쫄순이, 순두부쫄면",http://10000recipe.com/recipe/6989285
...,...,...
395,양송이 치즈구이,http://10000recipe.com/recipe/6835653
396,"채쳐서 바로 무치는 초스피드 밑반찬, 상큼한 양배추 생채",http://10000recipe.com/recipe/6882531
397,백종원 맑은 오징어무국 끓이기,http://10000recipe.com/recipe/6995719
398,매콤함이 파바박 집밥백선생♥꽈리고추삼겹살볶음,http://10000recipe.com/recipe/6885922


In [8]:
def get_ingredients(url):
    try: 
        response = requests.get(url, verify=False)
        soup = bs(response.text)    
        tmp = soup.find('div', 'ready_ingre3')
        ingredients = [i.text.split()[0] for i in tmp.find_all('li')]
        return ingredients
    except:
        return None

In [9]:
url = all_page.loc[0, '링크']
get_ingredients(url)

['밥', '김', '오이', '소금', '후추', '깨', '고추장', '된장', '다진마늘', '참기름', '깨']

In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

all_page['재료'] = all_page['링크'].progress_map(get_ingredients)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
# 400개 처리 Colab에서 20분 가량 소요

In [11]:
all_page  # 링크를 통해 요리 재료 수집 완료

,요리,링크,재료
0,오이 좋아한다면 통오이김밥,http://10000recipe.com/recipe/7001234,"[밥, 김, 오이, 소금, 후추, 깨, 고추장, 된장, 다진마늘, 참기름, 깨]"
1,버섯으로 관자 느낌 내는 방법! 새송이버섯간장버터구이 만들기,http://10000recipe.com/recipe/6912734,"[새송이버섯, 버터, 어린잎채소, 간장, 맛술, 올리고당]"
2,아삭아삭한 고추된장박이,http://10000recipe.com/recipe/4030952,"[풋고추, 된장, 다진, 송송썰은, 참기름, 통깨, 매실원액, 설탕]"
3,달콤 짭조름♡ 떡 베이컨 간장조림♡,http://10000recipe.com/recipe/6843136,"[베이컨, 대파, 떡, 간장, 참기름, 물엿, 기름]"
4,"술술 들어가는 쫄순이, 순두부쫄면",http://10000recipe.com/recipe/6989285,"[순두부, 쫄면사리, 달걀, 양파, 대파, 다진마늘, 고춧가루, 물, 간장, 굴소스..."
...,...,...,...
395,양송이 치즈구이,http://10000recipe.com/recipe/6835653,"[양파, 노랑파프리카, 주황파프리카, 양송이, 토마토스파게티소스, 모짜렐라치즈, 파..."
396,"채쳐서 바로 무치는 초스피드 밑반찬, 상큼한 양배추 생채",http://10000recipe.com/recipe/6882531,"[양배추, 간마늘, 식초, 매실청, 꿀/물엿, 멸치액젓, 국간장(조선간장), 고추가..."
397,백종원 맑은 오징어무국 끓이기,http://10000recipe.com/recipe/6995719,"[손질, 무, 대파, 청양고추, 다진, 국간장, 액젓, 들기름, 물]"
398,매콤함이 파바박 집밥백선생♥꽈리고추삼겹살볶음,http://10000recipe.com/recipe/6885922,"[삼겹살, 꽈리고추, 청양고추, 대파, 설탕, 물, 다진마늘, 간장]"


In [ ]:
# 필요에 따라 코드 개선